In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import time
from django.utils import dateparse
from datetime import timedelta
from datetime import datetime
import sqlite3
import random

In [2]:
try:
    sqlite_connection = sqlite3.connect('sqlite_python1.db')
    cursor = sqlite_connection.cursor()
    print("База данных создана и успешно подключена к SQLite")
    sqlite_select_query = "select sqlite_version();"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Версия базы данных SQLite: ", record)
    cursor.close()
except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")

База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.33.0',)]
Соединение с SQLite закрыто


In [3]:
def execute(sql):
    sqlite_connection = sqlite3.connect('sqlite_python1.db', timeout=10)
    cursor = sqlite_connection.cursor()
    cursor.execute(sql)
    sqlite_connection.commit()
    cursor.close()

def retexec(sql):
    sqlite_connection = sqlite3.connect('sqlite_python1.db', timeout=10)
    cursor = sqlite_connection.cursor()
    cursor.execute(sql)
    cur = cursor.fetchall()
    r = pd.DataFrame(cur)
    if cursor is not None and isinstance (cursor, sqlite3.Cursor):
        colnames = [row[0] for row in cursor.description]
        if len(r.columns) == len(colnames):
            r.columns = colnames
    sqlite_connection.commit()
    cursor.close()
    return r

def fill_table_many(table_name, dbs, datecolumns):
    start = datetime.now()
    data = dbs.copy()
    for c in data.columns:
        data[c] = data[c].apply(str)
    ve = [x.upper() for x in data.columns]
    vc = '", "'.join(ve)
    vc = '("'+vc+'")'
    vd = '", :"'.join(ve)
    vd = '(:"'+vd+'")'    
    ve = ['?' for x in data.columns]
    vc = ', '.join(ve)
    vc = '('+vc+')'
    sql = ("""insert into """+table_name+""" values """+vc)
    result = []
    for j in tqdm(range(len(data))):
        res = [data.loc[j, c] for c in data.columns]
        res = tuple(i for i in res)
        result.append(res)
    try:
        connection = sqlite3.connect('sqlite_python1.db', timeout=10)
        cur = connection.cursor()
        cur.executemany(sql,  result)
        connection.commit()
        cur.close()
    except:
        print(cur.rowcount)
    finish = datetime.now()
    t = '{}'.format(finish-start)
    print(t)

def create_table(table_name, ds, datecolumns):
    data = ds.copy()
    vc = []
    cat_col = [c for c in data.columns if data[c].dtype.name == 'object' if c not in datecolumns]
    num_col = [c for c in data.columns if data[c].dtype.name != 'object' if c not in datecolumns]
    for c in datecolumns:
        vc.append('"' +  c.upper() + ' TEXT"')
    for c in cat_col:
        vc.append('"' +  c.upper() + ' TEXT"')
    for c in num_col:
        vc.append('"' +  c.upper() + ' INTEGER"')
    vc = ', '.join(vc)
    sql = """create table """ +table_name+""" (  """ + vc +"""  )"""
    execute(sql)
    
def create_one_row(dt_start):
    v = ['"'+dt_start+'"']
    v.append('"'+str(dateparse.parse_datetime(dt_start).timestamp())+'"')
    v.append(int(random.choice(['1' for x in range(50)]+['2' for x in range(10)])))
    v.append(int(random.choice([str(x) for x in range(75)])))
    v.append(int(random.choice([str(x) for x in range(10,70)])))
    v.append('"'+random.choice(['Mobile','Desktop'])+'"')   
    v.append(int(random.choice([str(x) for x in range(10000)])))
    v.append(int(random.choice([str(x) for x in range(23)])))
    v.append(int(random.choice([str(x) for x in range(50000)])))
    v.append(int(random.choice([str(x) for x in range(90000)])))
    v.append(str(i))
    return v

## Создание таблицы events

### 1. Подготовка полей таблицы

In [4]:
dct = {'string':'TEXT', 'int':'INTEGER', 'date':'TEXT', 'datetime':'TEXT', 'boolean':'INTEGER'}

fields = 'date;timestamp;event_type;geo;age;device_type;Banner_id;Campaign_id;Amount;Pad_id;Event_id'
fields = fields.split(';')

types = 'date;datetime;int;int;int;string;int;int;int;int;int'
types = types.split(';')
types = [dct[x] for x in types]  
types[-1] = types[-1]+ ' PRIMARY KEY'
 
s = []
for i in range(len(fields)):
    s.append(' '.join([fields[i], types[i]]))
s = ', '.join(s)

### 2. Создание таблицы

In [5]:
table_name = 'events'
sql = """CREATE TABLE IF NOT EXISTS """+table_name+""" ( """+s+""");"""
execute(sql)

### 3. Заполнение таблицы синтезированными данными

In [6]:
dt = '01.02.2020 00:00:00'
dt = datetime.strptime(dt, "%d.%m.%Y %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

days = 30
dbs = []
for i in tqdm(range(days)):
    dt_start = (dateparse.parse_datetime(dt)+timedelta(seconds=i*3600*24)).strftime("%Y-%m-%d %H:%M:%S")
    n = 0
    if dt_start == '2020-02-29 00:00:00':
        n = 50
    for j in range(30+n):
        v = create_one_row(dt_start)
        dbs.append(v)

dbs = pd.DataFrame(dbs, columns = fields)
dbs['Event_id'] = [str(x) for x in range(len(dbs))]

fill_table_many(table_name, dbs, datecolumns=[])

100%|██████████| 950/950 [00:00<00:00, 17215.10it/s]

0:00:00.066808


In [7]:
dbs = retexec("""select * from events""")

In [9]:
dbs.head(10)

,date,timestamp,event_type,geo,age,device_type,Banner_id,Campaign_id,Amount,Pad_id,Event_id
0,"""2020-02-01 00:00:00""","""1580504400.0""",1,7,46,"""Mobile""",7094,11,11233,60262,0
1,"""2020-02-01 00:00:00""","""1580504400.0""",1,26,54,"""Desktop""",9441,0,14674,76396,1
2,"""2020-02-01 00:00:00""","""1580504400.0""",1,59,16,"""Desktop""",7439,7,40777,61847,2
3,"""2020-02-01 00:00:00""","""1580504400.0""",1,7,14,"""Mobile""",8953,11,20504,20095,3
4,"""2020-02-01 00:00:00""","""1580504400.0""",1,27,30,"""Desktop""",6574,1,47211,43813,4
5,"""2020-02-01 00:00:00""","""1580504400.0""",1,31,31,"""Desktop""",8442,16,25909,65012,5
6,"""2020-02-01 00:00:00""","""1580504400.0""",1,17,65,"""Mobile""",2446,6,22275,47367,6
7,"""2020-02-01 00:00:00""","""1580504400.0""",2,20,46,"""Mobile""",3449,10,40104,89043,7
8,"""2020-02-01 00:00:00""","""1580504400.0""",1,22,12,"""Desktop""",8770,3,39135,69204,8
9,"""2020-02-01 00:00:00""","""1580504400.0""",1,18,62,"""Mobile""",2738,15,5061,82634,9


## Создание таблицы campaigns

In [10]:
fields = 'id;is_active;name;format'
fields = fields.split(';')

types = 'int;boolean;string;string'
types = types.split(';')
types = [dct[x] for x in types]
types[0] = types[0]+ ' PRIMARY KEY'

descr = 'Идентификатор рекламной кампании;Флаг активности кампании;Имя кампании;\
"Формат рекламы. Принимает значения: Video Carousel Multiv Image"'
descr = descr.split(';')

s = []
for i in range(len(fields)):
    s.append(' '.join([fields[i], types[i]]))
s = ', '.join(s)

sql = """CREATE TABLE IF NOT EXISTS campaigns ( """+s+""");"""
execute(sql)

In [11]:
for i in tqdm(range(25)):
    v = [i]
    v.append(int(random.choice(['0','1'])))
    v.append('"'+random.choice(['first','second','third','fourth'])+'"')    
    v.append('"'+random.choice(['Video','Carousel','Multiv','Image'])+'"')
    s2 = ','.join([str(x) for x in v])
    sql = """INSERT INTO campaigns ("""+', '.join(fields)+""")  VALUES  ("""+s2+""")"""
    execute(sql)

100%|██████████| 25/25 [00:00<00:00, 1026.91it/s]


In [13]:
dba = retexec("""select * from campaigns""")

In [14]:
dba.head(10)

,id,is_active,name,format
0,0,1,first,Multiv
1,1,0,fourth,Image
2,2,1,fourth,Carousel
3,3,1,fourth,Image
4,4,0,fourth,Carousel
5,5,0,second,Image
6,6,1,third,Video
7,7,1,third,Video
8,8,1,first,Multiv
9,9,1,first,Video


### Задание 1.

29.02.2020 по отношению к предыдущей неделе выросли показы мобильных пользователей. 

Необходимо проанализировать, изменилась ли структура показов по возрастным группам.

Напишите запрос, который подготовит данные для проведения такого анализа. 

Пользователей разбейте на 4 группы:
* до 18 лет
* с 18 до 60 лет
* старше 60 лет
* пользователи, чей возраст не известен

Структура итоговой таблицы:
* дата
* возрастная группа
* количество показов

In [15]:
db1 = retexec("""
   with e as
    (select *,
      case
        when (age < 18) then "<-18"
        when (age >= 18 and age < 60) then "18-60"
        when (age >= 60) then "60+"
        when (age is null) then "undefined"
      end age_d
    from events
    order by age)
        
    select date "дата", age_d "возрастная группа", count(*) "количество показов" from e  
    where event_type = 1 and device_type in ('"Mobile"')
    and date >= ('"2020-02-22 00:00:00"') and date <= ('"2020-02-29 00:00:00"')
    group by date, age_d, event_type  
    order by age_d, date  
        """)

In [16]:
db1

,дата,возрастная группа,количество показов
0,"""2020-02-22 00:00:00""",18-60,16
1,"""2020-02-23 00:00:00""",18-60,10
2,"""2020-02-24 00:00:00""",18-60,10
3,"""2020-02-25 00:00:00""",18-60,9
4,"""2020-02-26 00:00:00""",18-60,13
5,"""2020-02-27 00:00:00""",18-60,7
6,"""2020-02-28 00:00:00""",18-60,10
7,"""2020-02-29 00:00:00""",18-60,23
8,"""2020-02-22 00:00:00""",60+,3
9,"""2020-02-23 00:00:00""",60+,2


### Задание 2.

В феврале 2020 года по отношению к январю 2020 произошло снижение CTR 

(Click-Through Rate, отношение количества кликов (2) по объявлению к количеству его показов (1)). 

Вам нужно проанализировать причины снижения. Для этого:

Посчитайте CTR в разрезе формата рекламы за эти два периода. 

Структура итоговой таблицы:
* период (январь / февраль)
* формат рекламы
* CTR

In [25]:
db2 = retexec("""
 with c as (

    with b as (
        with a as      (
        select e.event_type, c.format, CAST(substr(date,7,2) AS INTEGER) month from events e
        left join (select id, format from campaigns) c on c.id = e.Campaign_id
                       )
        select month, format, event_type, count(event_type) cnt from a
        group by month, event_type, format
              )
    select
      s.month   as month,
      s.format  as format,
      CAST(s.cnt as REAL)      as show,
      CAST(e.cnt as REAL)      as click

    from (select month, event_type, format, cnt from b where event_type=1) as s
    left join (select month, event_type, format, cnt from b where event_type=2) as e
        on s.format = e.format and s.month = e.month
    group by s.month, s.cnt
    order by s.month, s.cnt
          )
    select month, format, (click / show) CTR from c
    order by month, CTR
             """)

In [26]:
db2

,month,format,CTR
0,2,Carousel,0.146341
1,2,Multiv,0.193182
2,2,Image,0.215470
3,2,Video,0.250000
4,3,Video,0.142857
5,3,Image,0.166667
6,3,Carousel,0.250000
7,3,Multiv,0.666667


### Задание 3.

Посчитайте CTR в разрезе площадок. 

В итоговую таблицу выведите топ-5 площадок, на которых был самый низкий CTR в феврале 2020 года. 

Структура итоговой таблицы:
* идентификатор площадки
* CTR за январь 2020 года
* CTR за февраль 2020 года

In [31]:
db3 = retexec("""

 with c as (

     with b as (
        with a as (select geo, CAST(substr(date,7,2) AS INTEGER) month, e.event_type from events e)
        select month, geo, event_type, count(event_type) cnt from a
        group by month, event_type, geo
               )
    select  
        s.month as month,
        s.geo as geo,
        CAST(s.cnt as REAL)  as show,
        CAST(e.cnt as REAL)  as click
    from (select month, event_type, geo, cnt  from b where event_type=1 ) as s
    left join (select month, event_type, geo, cnt from b where event_type=2 ) as e
        on s.geo = e.geo and s.month = e.month
    group by s.month, s.cnt
    order by s.month, s.cnt
        )
    select month, geo, (click / show) CTR from c
    where CTR is not null
    order by CTR
    LIMIT 5;
         """)

In [32]:
db3

,month,geo,CTR
0,2,2,0.066667
1,2,11,0.083333
2,2,0,0.100000
3,2,4,0.181818
4,2,1,0.200000
